# Downloading Data

I load the CSV file that we saved in the previous assignment, containing the monthly log returns for the 10 companies.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
stocks_prices=pd.read_csv("monthly_returns_stocks10.csv") # I load the csv file that we saved in the previous assignment

stocks_prices['Date'] = pd.to_datetime(stocks_prices['Date'], format='%m/%Y')

stocks_prices = stocks_prices.set_index('Date')
stocks_prices.index = stocks_prices.index.strftime('%Y-%m')
stocks_prices

,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
Date,,,,,,,,,,
2000-01,-0.093035,-0.074365,-0.192907,-0.023270,-0.101497,0.110528,-0.032902,-0.281030,-0.097080,-0.008818
2000-02,0.473232,0.014013,0.647329,-0.158405,-0.089477,0.047244,-0.178897,0.049899,-0.142108,-0.056044
2000-03,0.001099,0.058602,0.079995,0.022407,0.095161,-0.056886,0.200926,-0.443044,0.038858,0.003874
2000-04,-0.176817,0.103173,0.091662,-0.040940,0.120596,0.146820,0.076154,-0.537581,0.154064,-0.024449
2000-05,-0.177001,0.054301,-0.068503,0.179674,0.020835,-0.028959,0.056292,-0.375043,0.020096,-0.003524
...,...,...,...,...,...,...,...,...,...,...
2022-08,-0.040726,-0.053462,-0.085545,0.059051,0.012374,0.123921,0.039061,-0.058180,-0.104138,-0.119802
2022-09,-0.053656,-0.059091,-0.305259,-0.088450,-0.073160,-0.118766,-0.055714,-0.116897,-0.150143,-0.118085
2022-10,0.131190,0.026987,0.146124,0.186869,0.066322,0.152271,0.147161,0.095061,0.125910,0.129598


In [80]:
stocks_prices.shift(-1)

,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
Date,,,,,,,,,,
2000-01,0.473232,0.014013,0.647329,-0.158405,-0.089477,0.047244,-0.178897,0.049899,-0.142108,-0.056044
2000-02,0.001099,0.058602,0.079995,0.022407,0.095161,-0.056886,0.200926,-0.443044,0.038858,0.003874
2000-03,-0.176817,0.103173,0.091662,-0.040940,0.120596,0.146820,0.076154,-0.537581,0.154064,-0.024449
2000-04,-0.177001,0.054301,-0.068503,0.179674,0.020835,-0.028959,0.056292,-0.375043,0.020096,-0.003524
2000-05,0.001619,0.086837,0.137672,-0.187117,-0.023823,0.042788,-0.112573,0.549756,-0.003558,-0.031114
...,...,...,...,...,...,...,...,...,...,...
2022-08,-0.053656,-0.059091,-0.305259,-0.088450,-0.073160,-0.118766,-0.055714,-0.116897,-0.150143,-0.118085
2022-09,0.131190,0.026987,0.146124,0.186869,0.066322,0.152271,0.147161,0.095061,0.125910,0.129598
2022-10,0.113509,0.083721,0.079719,0.009599,0.088782,0.100302,0.105454,0.071322,0.103277,0.013071


# Section 1

# Question 1

- Choose the estimation window to be T est = 60 months of monthly returns. Call this the estimation sample. Use the estimation sample to compute the following two portfolio strategies:
     - a.  mean-variance portfolio with nonnegativity constraints on the weights (when a risk-free rate is available, and set this rate to 0); we will refer to this portfolio as “MVP-C.”;
    - b. global minimum variance (GMV) portfolio with nonnegativity constraints; we will refer to this portfolio as “GMV-C”.
    
- For each of the two portfolios, rescale the weights in the risky assets so that they sum to 1; that is, you are “fully invested” in just the risky assets.

The mean-variance portfolio is the one that maximizes the sharpe ratio. I use minimize function from scipy.

In [3]:
import numpy as np
from scipy.optimize import minimize

def objective_function_MVP(weights, returns):
    # Calculate mean and standard deviation of portfolio returns
    portfolio_return = weights.T @ returns.mean() # because r=0
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))

    # Calculate the negative Sharpe ratio (since we want to maximize it)
    sharpe_ratio = -portfolio_return / portfolio_volatility

    return sharpe_ratio

def maximize_sharpe_ratio(returns):
    num_assets = returns.shape[1]

    # Set initial weights (equally weighted)
    initial_weights = np.ones(num_assets) / num_assets

    # Set optimization bounds (weights>=0)
    bounds = tuple((0, 999) for asset in range(num_assets))

    # Set optimization constraint (sum of weights equals 1)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    # Minimize the negative Sharpe ratio to maximize the Sharpe ratio
    result = minimize(objective_function_MVP, initial_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x


T_est=60
estimation_sample=stocks_prices[:T_est]
sharpe_max_weights = maximize_sharpe_ratio(estimation_sample)

print("MVP-C weights:")
pd.Series(sharpe_max_weights, index=estimation_sample.mean().index)

MVP-C weights:


A       2.996826e-14
ABT     2.090876e-01
ADBE    2.930084e-02
ADM     1.677535e-01
ADP     1.092996e-14
AES     3.097998e-14
AFL     1.476462e-01
AKAM    4.221426e-14
AOS     7.797059e-02
MMM     3.682413e-01
dtype: float64

In [4]:
sharpe_max_weights.sum()

1.0

The GMV portfolio is the one that minimizes the variance.

In [5]:
import numpy as np
from scipy.optimize import minimize

def objective_function_GMV(weights, cov_matrix):
    # Calculate portfolio variance
    portfolio_variance = weights.T @ cov_matrix @ weights
    return portfolio_variance

def minimize_variance_portfolio(returns):
    num_assets = returns.shape[1]

    # Set initial weights (equally weighted)
    initial_weights = np.ones(num_assets) / num_assets

    # Calculate the covariance matrix of returns
    cov_matrix = np.cov(returns, rowvar=False)

    # Set optimization bounds (weights >= 0)
    bounds = tuple((0, None) for asset in range(num_assets))

    # Set optimization constraint (sum of weights equals 1)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    # Minimize the portfolio variance to find the global minimum variance portfolio
    result = minimize(objective_function_GMV, initial_weights, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x

T_est = 60
estimation_sample = stocks_prices[:T_est]
min_variance_weights = minimize_variance_portfolio(estimation_sample)

print("GMV-C weights:")
print(pd.Series(min_variance_weights, index=estimation_sample.cov().index))

GMV-C weights:
A       0.007078
ABT     0.293203
ADBE    0.013353
ADM     0.047752
ADP     0.063562
AES     0.002308
AFL     0.136072
AKAM    0.000000
AOS     0.104734
MMM     0.331938
dtype: float64


In [6]:
min_variance_weights.sum()

1.0

# Section 2

# Question 2

- Now use a rolling window of T est = 60 months to estimate the portfolio weights for the two strategies listed above for each of the T − T est months. That is, repeat the calculations of the previous question for all the dates after the first 60 months.

In [7]:
T_est = 60
num_months = len(stocks_prices)
columns = stocks_prices.columns

# Create an empty DataFrame to store the results
MVP_C = pd.DataFrame(index=stocks_prices.index[T_est+1:], columns=columns)

# Iterate through each date after the first 60 months
for i in range(T_est+1, num_months):
    estimation_sample = stocks_prices[i - T_est:i]
    sharpe_max_weights = maximize_sharpe_ratio(estimation_sample)
    #print(sharpe_max_weights.sum())

    # Store the results in the DataFrame
    MVP_C.loc[stocks_prices.index[i]] = sharpe_max_weights

# Display the resulting DataFrame
print("MVP-C Weights:")
MVP_C

MVP-C Weights:


,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
Date,,,,,,,,,,
2005-02,0.0,0.213342,0.043428,0.202767,0.0,0.0,0.128738,0.0,0.071781,0.339945
2005-03,0.0,0.168189,0.0,0.231046,0.0,0.0,0.214892,0.0,0.091008,0.294864
2005-04,0.0,0.166435,0.0,0.249754,0.0,0.0,0.120677,0.0,0.10736,0.355774
2005-05,0.0,0.213849,0.0,0.11649,0.0,0.0,0.229996,0.0,0.100017,0.339648
2005-06,0.0,0.184737,0.0,0.094541,0.0,0.0,0.22216,0.0,0.123165,0.375396
...,...,...,...,...,...,...,...,...,...,...
2022-08,0.0,0.411831,0.139813,0.254305,0.13185,0.0,0.0,0.062201,0.0,0.0
2022-09,0.0,0.314486,0.047794,0.300658,0.272209,0.002459,0.0,0.062393,0.0,0.0
2022-10,0.0,0.369745,0.0,0.309063,0.321192,0.0,0.0,0.0,0.0,0.0


In [8]:
T_est = 60
num_months = len(stocks_prices)
columns = stocks_prices.columns

# Create an empty DataFrame to store the results
GMV_C = pd.DataFrame(index=stocks_prices.index[T_est+1:], columns=columns)

# Iterate through each date after the first 60 months
for i in range(T_est+1, num_months):
    estimation_sample = stocks_prices[i - T_est:i]
    min_variance_weights = minimize_variance_portfolio(estimation_sample)

    # Store the results in the DataFrame
    GMV_C.loc[stocks_prices.index[i]] = min_variance_weights

print("GMV-C Weights:")
GMV_C

GMV-C Weights:


,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
Date,,,,,,,,,,
2005-02,0.0,0.292476,0.025156,0.0672,0.079995,0.0,0.14991,0.0,0.087041,0.298221
2005-03,0.0,0.261853,0.0,0.084099,0.100791,0.0,0.1819,0.0,0.088278,0.28308
2005-04,0.0,0.253132,0.0,0.081098,0.109304,0.0,0.210526,0.0,0.088019,0.25792
2005-05,0.0,0.269731,0.0,0.036088,0.129032,0.0,0.232763,0.0,0.109271,0.223113
2005-06,0.0,0.270216,0.0,0.049023,0.128526,0.0,0.231617,0.0,0.104348,0.21627
...,...,...,...,...,...,...,...,...,...,...
2022-08,0.0,0.229102,0.005553,0.178453,0.152051,0.028276,0.106133,0.262698,0.0,0.037734
2022-09,0.0,0.22163,0.000372,0.183726,0.154044,0.043737,0.111816,0.255375,0.0,0.0293
2022-10,0.0,0.258337,0.0,0.181581,0.173215,0.002781,0.114995,0.260173,0.0,0.008919


# Section 3

# Question 3

-  Use the time-series of portfolios weights for each of the two portfolio strategies, to compute the out-of-sample portfolio returns. That is, for each of the two portfolio strategies that you estimate at each date t, compute its out-of-sample return in month t + 1.

In [12]:
T_est = 60
num_months = len(stocks_prices)
columns = stocks_prices.columns

# Create an empty DataFrame to store the results
MVP_C = pd.DataFrame(index=stocks_prices.index[T_est+1:], columns=columns)

# Iterate through each date after the first 60 months
for i in range(T_est+1, num_months):
    estimation_sample = stocks_prices[i - T_est:i]
    sharpe_max_weights = maximize_sharpe_ratio(estimation_sample)
    MVP_C.loc[stocks_prices.index[i]] = sharpe_max_weights

# Compute out-of-sample portfolio returns
out_of_sample_returns = (stocks_prices.shift(-1).loc[MVP_C.index] * MVP_C).sum(axis=1)
    
# Create a DataFrame for out-of-sample returns
out_of_sample_returns_mvp = pd.DataFrame(out_of_sample_returns, columns=['Out of Sample MVP-C Return'])
out_of_sample_returns_mvp

,Out of Sample MVP-C Return
Date,
2005-02,0.01974
2005-03,-0.077737
2005-04,0.038538
2005-05,-0.01382
2005-06,0.024621
...,...
2022-08,-0.106426
2022-09,0.096014
2022-10,0.062438


In [9]:
T_est = 60
mvp_weights_list = []
out_of_sample_returns_list = []
for i in range(T_est+1, len(stocks_prices)-1):
    returns_window = stocks_prices.iloc[i - T_est:i]
    sharpe_max_weights = maximize_sharpe_ratio(returns_window)
    mvp_weights_list.append(sharpe_max_weights)
    returns_next_month = stocks_prices.iloc[i + 1]
    out_of_sample_return = np.dot(sharpe_max_weights, returns_next_month)
    out_of_sample_returns_list.append(out_of_sample_return)

out_of_sample_returns_mvp2 = pd.DataFrame(out_of_sample_returns_list, columns=['Out of Sample MVP-C Return'])
out_of_sample_returns_mvp2

,Out of Sample MVP-C Return
0,0.019740
1,-0.077737
2,0.038538
3,-0.013820
4,0.024621
...,...
209,-0.019361
210,-0.106426
211,0.096014
212,0.062438


In [10]:
T_est = 60
num_months = len(stocks_prices)
columns = stocks_prices.columns

# Create an empty DataFrame to store the results
GMV_C = pd.DataFrame(index=stocks_prices.index[T_est+1:], columns=columns)

# Iterate through each date after the first 60 months
for i in range(T_est+1, num_months):
    estimation_sample = stocks_prices[i - T_est:i]
    min_variance_weights = minimize_variance_portfolio(estimation_sample)
    GMV_C.loc[stocks_prices.index[i]] = min_variance_weights

# Compute out-of-sample portfolio returns
out_of_sample_returns = (stocks_prices.shift(-1).loc[GMV_C.index] * GMV_C).sum(axis=1)
    
# Create a DataFrame for out-of-sample returns
out_of_sample_returns_gmv = pd.DataFrame(out_of_sample_returns, columns=['Out of Sample GMV-C Return'])
out_of_sample_returns_gmv

,Out of Sample GMV-C Return
Date,
2005-02,0.020452
2005-03,-0.031319
2005-04,0.02007
2005-05,-0.018304
2005-06,0.018862
...,...
2022-08,-0.086577
2022-09,0.101773
2022-10,0.069828


# Section 4

# Question 4

- Now, compute the Sharpe ratio of the out-of-sample returns for the two portfolio strategies. Which strategy has the higher Sharpe ratio?

In [13]:
sharpe_ratio_mvp = out_of_sample_returns_mvp.mean() / out_of_sample_returns_mvp.std() # assuming rf=0
sharpe_ratio_gmv = out_of_sample_returns_gmv.mean() / out_of_sample_returns_gmv.std()
print("\nSharpe Ratio MVP:")
print(sharpe_ratio_mvp)

print("\nSharpe Ratio GMV:")
print(sharpe_ratio_gmv)


Sharpe Ratio MVP:
Out of Sample MVP-C Return    0.172143
dtype: object

Sharpe Ratio GMV:
Out of Sample GMV-C Return    0.175414
dtype: object


We see that both strategies have roughly the same sharpe ratio, but GMV Sharape Ratio is slightly above the MVP one.